In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import urllib
import sched, time

In [3]:
def do_processing(img):
    cv2.imwrite( "website/public/images/dashboard/hackdtu1.jpg", img )
    #load a frame as int16 array to avoid overflow etc. when subtracting
    cv2.imshow("Image1",img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Image",img)
    # Converts images from BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    lower_red = np.array([0,100,100])
    upper_red = np.array([10,255,255])

    mask = cv2.inRange(hsv, lower_red, upper_red)

    res = cv2.bitwise_and(img,img, mask= mask)
    #cv2.imshow('frame',frame)
    #cv2.imshow('mask',mask)
    cv2.imshow("Image",res)
    img_b = res[:, :, 0]
    img_g = res[:, :, 1]
    img_r = res[:, :, 2]

    #for a red stripe, the red component should be much bigger than the rest
    res_br = img_r - img_b
    res_gr = img_r - img_g
    res = np.maximum(res_br, res_gr)
    res[res < 0] = 0
    res = res.astype(np.uint8)  #convert back to uint8 for findContours etc.
    cv2.imshow("Image",res)
    #find contours and count them
    imt, contours, hierarchy = cv2.findContours(res, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # 2.2 Sort contours by area in decreasing order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    #gives the correct output of 5
#     print(len(contours))

    #alternatively, count connected components with this:
    
    ans = 0
    for c in contours:
        area = cv2.contourArea(c) 
        if area > 1500 and area < 4000:
            ans = ans+1
            p = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * p, True)
            cv2.drawContours(img, [approx], -1, (255, 255, 255), 5)
        if area < 1500:
            break
    print(ans)
    cv2.imshow("Image2",img)

In [4]:
stream=urllib.urlopen('http://10.60.24.112:8080/stream.mjpeg')
bytes=''
while True:
    bytes+=stream.read(1024)
    a = bytes.find('\xff\xd8')
    b = bytes.find('\xff\xd9')
    if a!=-1 and b!=-1:
        jpg = bytes[a:b+2]
        bytes= bytes[b+2:]
        frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8),cv2.IMREAD_COLOR)
        do_processing(frame)
        if cv2.waitKey(1) ==27:
            exit(0)
        
# Destroys all of the HighGUI windows.
cv2.destroyAllWindows()

1
1
1
1
1


KeyboardInterrupt: 

In [4]:
img = cv2.imread('ima.png')

In [6]:
 cv2.imshow("Image",img)

error: OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/highgui/src/window.cpp:356: error: (-215) size.width>0 && size.height>0 in function imshow
